## First we install Python packages and initialize our globals

In [0]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/339.6 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━ 143.4/339.6 kB 4.6 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.0 MB ? eta -:--:--
     ━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.6/2.0 MB 16.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 1.2/2.0 MB 17.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 MB 19.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/200.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 22.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
# load table into spark dataframe
cows_bff = spark.read.table("db.cows_bff")

## Some graphs and charts

In [0]:
display(cows_bff.limit(10))

cow_name,meal_start,meal_end,meal_start_time,meal_end_time,duration,date,day
Butterscotch,32463,33042,9:01:03,9:10:42,579,2023-05-20T00:00:00Z,Saturday
Hershey,31916,34307,8:51:56,9:31:47,2391,2023-05-20T00:00:00Z,Saturday
Mocha,32519,34494,9:01:59,9:34:54,1975,2023-05-20T00:00:00Z,Saturday
Cocoa,32519,34494,9:01:59,9:34:54,1975,2023-05-20T00:00:00Z,Saturday
Nutella,32304,34452,8:58:24,9:34:12,2148,2023-05-20T00:00:00Z,Saturday
Brandy,34521,34756,9:35:21,9:39:16,235,2023-05-20T00:00:00Z,Saturday
Peaches,32060,32960,8:54:20,9:09:20,900,2023-05-20T00:00:00Z,Saturday
Marshmallow,31971,34139,8:52:51,9:28:59,2168,2023-05-20T00:00:00Z,Saturday
Popcorn,32256,33322,8:57:36,9:15:22,1066,2023-05-20T00:00:00Z,Saturday
Muffin,33552,33848,9:19:12,9:24:08,296,2023-05-20T00:00:00Z,Saturday


## Lets compute meal overlap

In [0]:
from pyspark.sql.functions import udf, countDistinct, sum, col
from pyspark.sql.types import IntegerType

calculate_time_overlap = lambda start_interval_1, end_interval_1, start_interval_2, end_interval_2: \
    0 if end_interval_1 <= start_interval_2 or end_interval_2 <= start_interval_1 else \
    max(end_interval_1, end_interval_2) - min(start_interval_1, start_interval_2) 

calculate_time_overlap_udf = udf(calculate_time_overlap, IntegerType())

cow1 = cows_bff\
    .withColumnRenamed('cow_name','cow1')\
    .withColumnRenamed('meal_start','meal_start1')\
    .withColumnRenamed('meal_end','meal_end1')\
    .withColumnRenamed('date','date1')\
    .select('cow1','meal_start1','meal_end1','date1')


cow2 = cows_bff\
    .withColumnRenamed('cow_name','cow2')\
    .withColumnRenamed('meal_start','meal_start2')\
    .withColumnRenamed('meal_end','meal_end2')\
    .withColumnRenamed('date','date2')\
    .select('cow2','meal_start2','meal_end2','date2')

df = cow1.crossJoin(cow2)\
  .where((cow1.cow1 != cow2.cow2) & (cow1.date1 == cow2.date2))

df = df\
  .withColumn('overlap', calculate_time_overlap_udf("meal_start1", "meal_end1", "meal_start2", "meal_end2"))\
  .select('cow1','cow2','date1','overlap')

df = df.groupBy('cow1', 'cow2').agg(sum('overlap').alias('total_overlap'), countDistinct('date1').alias('distinct_days'))
df = df\
  .withColumn('avg_overlap', df.total_overlap / df.distinct_days)\
  .select('cow1','cow2','avg_overlap')\
  .withColumnRenamed('avg_overlap','distance')\
  .sort(col('cow1').asc(), col('cow2').asc())


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:714)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:430)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:430)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

# Display Heatmap

In [0]:
display(df)

Databricks visualization. Run in Databricks to view.

cow1,cow2,distance
Brandy,Buttercup,934.4615384615385
Brandy,Butterscotch,1107.6153846153845
Brandy,Cocoa,1004.7692307692307
Brandy,Daisy,1359.0
Brandy,Dottie,1189.7692307692307
Brandy,Hershey,1103.0
Brandy,Magic,789.2307692307693
Brandy,Marshmallow,1120.3076923076924
Brandy,Mocha,1004.7692307692307
Brandy,Muffin,1291.0
